![Podmanlogo](Pictures/podman-logo.png)

# Podman storage

As explained through the Podman 101 workshop, once a container is deleted all the data stored in it is also deleted. This is a problem for certain applications that may require to persist data.
Think of a containerized database, at some point you will update the database version, to achieve this you'll delete your old version container and deploy the new version container. In this example all the data stored in the database would be lost. Then, how can application data be preserved? Podman provides some different options for adding persistent storage and manage volumes, we are going to explore them throught this section of the workshop.

## Store Data on Host Machine

Podman provides mechanisms to use container host machine storage by using volumes and bind mounts. It provides the following benefits:
 - Data stored in the container host machine persists across container deletions.
 - As you're not using the Copy On Write (COW) filesystem within the container, it will usually provide better write performance.
 - Data can be shared between multiple containers. A typical examples is when one container is accessing to the volume for read/write operations while a second container is accessing only for reads.
 - It's possible to use external storage as you can host data mounts over the network, for examples by using NFS.

There are two ways of managing storage: volumes and binds. Volumes are fully managed by Podman while binds are data mounts managed by the user.

Let's begin with binds, first we need to create a directory in which we will store our data.

In [2]:
!mkdir ~/my-persistent-storage

mkdir: cannot create directory ‘/home/ppreciad/my-persistent-storage’: File exists


Now, lets move into the new directory and create an index.html file that we will use later for our containerized webserver.

In [12]:
!cd ~/my-persistent-storage
!echo "This message is stored in the container host, in the directory ~/my-persistent-storage" > ~/my-persistent-storage/index.html
!cat index.html

Now we can use an nginx web server container image without any storage mounted to it.

In [13]:
!podman run -it --rm -d -p 8080:80 --name my-web-server nginx

Resolved "nginx" as an alias (/home/ppreciad/.cache/containers/short-name-aliases.conf)
Trying to pull docker.io/library/nginx:latest...
Getting image source signatures
Copying blob e5fab51fcab0 [--------------] 0.0b / 1.2KiB (skipped: 0.0b = 0.00%)
Copying blob e5fab51fcab0 [=============>----------] 716.0b / 1.2KiB | 4.0 MiB/s
Copying blob af107e978371 [-------------] 0.0b / 27.8MiB (skipped: 0.0b = 0.00%)
Copying blob af107e978371 [---------------------------] 0.0b / 27.8MiB | 0.0 b/s
Copying blob 8fb6e3475860 [--------------] 0.0b / 626.0b (skipped: 0.0b = 0.00%)
Copying blob 8fb6e3475860 [----------------------------] 0.0b / 626.0b | 0.0 b/s
Copying blob 1581bea9f1d2 [-------------] 0.0b / 39.5MiB (skipped: 0.0b = 0.00%)
Copying blob 1581bea9f1d2 [---------------------------] 0.0b / 39.5MiB | 0.0 b/s
Copying blob 7bfc9d79c672 [--------------] 0.0b / 959.0b (skipped: 0.0b = 0.00%)
Copying blob 7bfc9d79c672 [----------------------------] 0.0b / 959.0b | 0.0 b/s
Copying blob 3ebd268a

Verify it's running and showing the default message.

In [14]:
!curl localhost:8080

<!DOCTYPE html>
<html>
<head>
<title>Welcome to nginx!</title>
<style>
html { color-scheme: light dark; }
body { width: 35em; margin: 0 auto;
font-family: Tahoma, Verdana, Arial, sans-serif; }
</style>
</head>
<body>
<h1>Welcome to nginx!</h1>
<p>If you see this page, the nginx web server is successfully installed and
working. Further configuration is required.</p>

<p>For online documentation and support please refer to
<a href="http://nginx.org/">nginx.org</a>.<br/>
Commercial support is available at
<a href="http://nginx.com/">nginx.com</a>.</p>

<p><em>Thank you for using nginx.</em></p>
</body>
</html>


We see the expected behaviour, it shows the default nginx web server file. During Podman 101 workshop we modified this message by creating a new custom container image. Now we are going to demonstrate how to use bind storage to achieve the same results.

In this case we will use the "-v" or "--volume" option in the command and then we will specify the directory to be mounted from the host and the directory in which to mount it inside the container. It has to be expressed in the "my-container-host-directory:my-container-mount-point:OPTIONS". The "OPTIONS" part is optional, but it's important to use the ":Z" argument if we have SElinux enabled in our system as it will the proper security context to our directory so our container can use the data within it.

Before we can test it we will stop the previous container:

In [18]:
!podman stop my-web-server

my-web-server


Now we can create our new webserver with the bind storage:

In [19]:
!podman run -it --rm -d -p 8080:80 --name my-web-server -v ~/my-persistent-storage:/usr/share/nginx/html:Z nginx

eade4b2f26c3a869e61635a1723d5193942a0acc431cfa4cb96af1aa293b6830


Check the message that it's showing:

In [20]:
!curl localhost:8080

This message is stored in the container host, in the directory ~/my-persistent-storage


With this we have demonstrated how easy it is to use bind mounts and reutilize the same container image with different outputs just by modifying the mount. This is a way of improve efficiency and save storage space as you don't need to create a dedicated container image for each application, in this case for each web server.

Before we proceed with volume mounts lets remove our recently created container.

In [21]:
!podman stop my-web-server

my-web-server


We've seen how to use binds, in that scenario the user was in responsible for creating a directory and mountin it to the container. With volumes it's Podman who manages the data mounts. You can manage volumes by using the podman volume command.

In [22]:
!podman volume create my-persistent-volume

my-persistent-volume


You can list all of your volumes

In [23]:
!podman volume ls

DRIVER      VOLUME NAME
local       ee17c8e83c840d1d6fc6caa5e7963ccebe18ffe5d90cb9bd14910eae73cea8fb
local       my-persistent-volume


And even get detailed information of them:

In [25]:
!podman inspect my-persistent-volume

[
     {
          "Name": "my-persistent-volume",
          "Driver": "local",
          "Mountpoint": "/home/ppreciad/.local/share/containers/storage/volumes/my-persistent-volume/_data",
          "CreatedAt": "2023-12-19T10:52:27.494460182+01:00",
          "Labels": {},
          "Scope": "local",
          "Options": {},
          "MountCount": 0,
          "NeedsCopyUp": true,
          "NeedsChown": true,
          "LockNumber": 0
     }
]


When you work with volumes in rootless containers, Podman stores the data in the $HOME/.local/share/containers/storage/volumes/ directory.

You can mount a volume using the same convention as we used with binds, but instead of specifying a local directory you just need to use the volume name:

In [85]:
!podman run -it --rm -d --name my-persistent-data-container -v my-persistent-volume:/home fedora

e36e022f1c97f801b7d259b6b623c22c34ce73b13a9559f762aa07608ae8cf6a


As Podman is managing the volume you do not need to add the SElinux permissions.

As you can see, we have mounted our volume to the /tmp directory. Lets check whats inside it:

In [86]:
!podman exec my-persistent-data-container ls /home

data.txt


The directory is empty, just as expected. Now we will add some information to it:

In [87]:
!podman exec my-persistent-data-container rm /home/data.txt
!podman exec my-persistent-data-container echo "This is some volume data" >> /home/data.txt
!podman exec my-persistent-data-container cat /home/data.txt

/bin/bash: line 1: /home/data.txt: Permission denied
cat: /home/data.txt: No such file or directory


You see the previous command failed. This is because the nginx container does not have "echo" package installed. Many of the commercial container images use a very limited subset of commands inside the container, this brings two main benefits:
- Reduce the storage used by the container image
- Reduce the attack surface. If an attacker gains access to the container they'll have a very limited set of tools to attack other systems or modify the one they got access to.

This sounds good, but without this tools how can I modify the data within my volumes? There are different ways of achieving that, one of the most common ones is exporting and importing data.

First we would need to export the information of our volume.

In [83]:
!podman stop my-persistent-data-container

WARN[0010] StopSignal SIGTERM failed to stop container my-persistent-data-container in 10 seconds, resorting to SIGKILL 
my-persistent-data-container




To create a volume called http-data, use the following command:

[user@host ~]$ podman volume create http-data
d721d941960a2552459637da86c3074bbba12600079f5d58e62a11caf6a591b5

You can inspect the volume by using the podman volume inspect command:

[user@host ~]$ podman volume inspect http-data
[
    {
        "Name": "http-data",
        "Driver": "local",
        "Mountpoint": "/home/user/.local/share/containers/storage/volumes/http-data/_data",
        "CreatedAt": "2022-07-12T17:10:12.709259987+02:00",
        "Labels": {},
        "Scope": "local",
        "Options": {}
    }
]

For rootless containers, Podman stores local volume data in the $HOME/.local/share/containers/storage/volumes/ directory.

To mount the volume into a container, refer to the volume by the volume name:

[user@host ~]$ podman run -p 8080:8080 --volume  http-data:/var/www/html \
  registry.access.redhat.com/ubi8/httpd-24:latest

Because Podman manages the volume, you do not need to configure SELinux permissions.
Exporting and Importing Data with Volumes

You can import data from a tar archive into an existing Podman volume by using the podman volume import VOLUME_NAME ARCHIVE_NAME command.

[user@host ~]$ podman volume import http_data web_data.tar.gz
...no output expected...

You can also export data from an existing Podman volume and save it as a tar archive on the local machine by using the podman volume export VOLUME NAME --output ARCHIVE_NAME command.

[user@host ~]$ podman volume export http_data --output web_data.tar.gz
...no output expected...





Volumes are data mounts managed by Podman. Bind mounts are data mounts managed by the user.

Both volumes and bind mounts can use the --volume (or -v) parameter.

--volume /path/on/host:/path/in/container:OPTIONS

In the preceding syntax, the :OPTIONS part is optional. Note that you can specify host paths by using absolute paths, such as /home/user/www, or relative paths, such as ./www, which refers to the www directory in the current working directory.

Use -v volume_name:/path/in/container to refer to a volume.

Alternatively, you can use the --mount parameter with the following syntax:

--mount type=TYPE,source=/path/on/host,destination=/path/in/container

The --mount parameter explicitly specifies the volume type, such as:

    bind for bind mounts.

    volume for volume mounts.

    tmpfs for creating memory-only, ephemeral mounts.

The --mount parameter is the preferred way of mounting directories in a container. However, because the -v parameter is still widely used, this course uses both styles.

Developers commonly use bind mounts for testing, or for mounting environment-specific files, such as property files. Because Podman manages the volume file system, use volumes to ensure consistent mount behavior across systems. Additionally, use volumes for advanced uses, such as mounting a remote volume over NFS.
Storing Data with Bind Mounts

Bind mounts can exist anywhere on the host system.

For example, to mount the /www directory on your host machine to the /var/www/html directory inside the container with the read-only option, use the following podman run command:

[user@host ~]$ podman run -p 8080:8080 --volume  /www:/var/www/html:ro \
  registry.access.redhat.com/ubi8/httpd-24:latest

Troubleshoot Bind Mounts

When you use bind mounts, you must configure file permissions and SELinux access manually. SELinux is an additional security mechanism used by Red Hat Enterprise Linux (RHEL) and other Linux distributions.

Consider the following bind mount example:

[user@host ~]$ podman run -p 8080:8080 --volume /www:/var/www/html \
  registry.access.redhat.com/ubi8/httpd-24:latest

By default, the Httpd process has insufficient permissions to access the /var/www/html directory. This can be a file permission issue or an SELinux issue.

To troubleshoot file permission issues, use the podman unshare command to execute the ls -l command. The podman unshare command executes provided Linux commands in a new namespace such as the one Podman creates for the container. This process maps user IDs as they are mapped in a new container, which is useful for troubleshooting user permissions.

[user@host ~]$ podman unshare ls -l /www/
total 4
-rw-rw-r--. 1 root root 21 Jul 12 15:21 index.html
[user@host ~]$ podman unshare ls -ld /www/
drwxrwxr-x. 1 root root 20 Jul 12 15:21 /www/

The previous example reveals that the /www/index.html directory is accessible to all users:

    The /www/index.html file provides read permissions to all users.

    The /www directory is viewed as owned by the root user and the root group in a new namespace.

    The /www directory provides the execute permissions to all users, which gives all users access to the directory contents.

This means that the file and directory permissions are correct in the bind mount.

To troubleshoot SELinux permission issues, inspect the /www directory SELinux configuration by running the ls command with the -Z option. Use the -d option to print only the directory information.

[user@host ~]$ ls -Zd /www
system_u:object_r:default_t:s0:c228,c359 /www

The output shows the SELinux context label system_u:object_r:default_t:s0:c228,c359, which has the default_t type. A container must have the container_file_t SELinux type to have access to the bind mount. SELinux is out of scope for this course.

To fix the SELinux configuration, add the :z or :Z option to the bind mount:

    Lower case z lets different containers share access to a bind mount.

    Upper case Z provides the container with exclusive access to the bind mount.

[user@host ~]$ podman run -p 8080:8080 --volume /www:/var/www/html:Z \
  registry.access.redhat.com/ubi8/httpd-24:latest

After adding the corresponding option, run the ls -Zd command and notice the right SELinux type.

[user@host ~]$ ls -Zd /www
system_u:object_r:container_file_t:s0:c240,c717 /www

The container_file_t SELinux type allows the container to access the bind mount files.
Important

Changing the SELinux label for system directories might lead to unexpected issues.
Storing Data with Volumes

Volumes let Podman manage the data mounts. You can manage volumes by using the podman volume command.

To create a volume called http-data, use the following command:

[user@host ~]$ podman volume create http-data
d721d941960a2552459637da86c3074bbba12600079f5d58e62a11caf6a591b5

You can inspect the volume by using the podman volume inspect command:

[user@host ~]$ podman volume inspect http-data
[
    {
        "Name": "http-data",
        "Driver": "local",
        "Mountpoint": "/home/user/.local/share/containers/storage/volumes/http-data/_data",
        "CreatedAt": "2022-07-12T17:10:12.709259987+02:00",
        "Labels": {},
        "Scope": "local",
        "Options": {}
    }



Volumes are data mounts managed by Podman. Bind mounts are data mounts managed by the user.

Both volumes and bind mounts can use the --volume (or -v) parameter.

--volume /path/on/host:/path/in/container:OPTIONS

In the preceding syntax, the :OPTIONS part is optional. Note that you can specify host paths by using absolute paths, such as /home/user/www, or relative paths, such as ./www, which refers to the www directory in the current working directory.

Use -v volume_name:/path/in/container to refer to a volume.

Alternatively, you can use the --mount parameter with the following syntax:

--mount type=TYPE,source=/path/on/host,destination=/path/in/container

The --mount parameter explicitly specifies the volume type, such as:

    bind for bind mounts.

    volume for volume mounts.

    tmpfs for creating memory-only, ephemeral mounts.

The --mount parameter is the preferred way of mounting directories in a container. However, because the -v parameter is still widely used, this course uses both styles.

Developers commonly use bind mounts for testing, or for mounting environment-specific files, such as property files. Because Podman manages the volume file system, use volumes to ensure consistent mount behavior across systems. Additionally, use volumes for advanced uses, such as mounting a remote volume over NFS.
Storing Data with Bind Mounts

Bind mounts can exist anywhere on the host system.

For example, to mount the /www directory on your host machine to the /var/www/html directory inside the container with the read-only option, use the following podman run command:

[user@host ~]$ podman run -p 8080:8080 --volume  /www:/var/www/html:ro \
  registry.access.redhat.com/ubi8/httpd-24:latest

Troubleshoot Bind Mounts

When you use bind mounts, you must configure file permissions and SELinux access manually. SELinux is an additional security mechanism used by Red Hat Enterprise Linux (RHEL) and other Linux distributions.

Consider the following bind mount example:

[user@host ~]$ podman run -p 8080:8080 --volume /www:/var/www/html \
  registry.access.redhat.com/ubi8/httpd-24:latest

By default, the Httpd process has insufficient permissions to access the /var/www/html directory. This can be a file permission issue or an SELinux issue.

To troubleshoot file permission issues, use the podman unshare command to execute the ls -l command. The podman unshare command executes provided Linux commands in a new namespace such as the one Podman creates for the container. This process maps user IDs as they are mapped in a new container, which is useful for troubleshooting user permissions.

[user@host ~]$ podman unshare ls -l /www/
total 4
-rw-rw-r--. 1 root root 21 Jul 12 15:21 index.html
[user@host ~]$ podman unshare ls -ld /www/
drwxrwxr-x. 1 root root 20 Jul 12 15:21 /www/

The previous example reveals that the /www/index.html directory is accessible to all users:

    The /www/index.html file provides read permissions to all users.

    The /www directory is viewed as owned by the root user and the root group in a new namespace.

    The /www directory provides the execute permissions to all users, which gives all users access to the directory contents.

This means that the file and directory permissions are correct in the bind mount.

To troubleshoot SELinux permission issues, inspect the /www directory SELinux configuration by running the ls command with the -Z option. Use the -d option to print only the directory information.

[user@host ~]$ ls -Zd /www
system_u:object_r:default_t:s0:c228,c359 /www

The output shows the SELinux context label system_u:object_r:default_t:s0:c228,c359, which has the default_t type. A container must have the container_file_t SELinux type to have access to the bind mount. SELinux is out of scope for this course.

To fix the SELinux configuration, add the :z or :Z option to the bind mount:

    Lower case z lets different containers share access to a bind mount.

    Upper case Z provides the container with exclusive access to the bind mount.

[user@host ~]$ podman run -p 8080:8080 --volume /www:/var/www/html:Z \
  registry.access.redhat.com/ubi8/httpd-24:latest

After adding the corresponding option, run the ls -Zd command and notice the right SELinux type.

[user@host ~]$ ls -Zd /www
system_u:object_r:container_file_t:s0:c240,c717 /www

The container_file_t SELinux type allows the container to access the bind mount files.
Important

Changing the SELinux label for system directories might lead to unexpected issues.



Storing Data with Volumes

Volumes let Podman manage the data mounts. You can manage volumes by using the podman volume command.

To create a volume called http-data, use the following command:

[user@host ~]$ podman volume create http-data
d721d941960a2552459637da86c3074bbba12600079f5d58e62a11caf6a591b5

You can inspect the volume by using the podman volume inspect command:

[user@host ~]$ podman volume inspect http-data
[
    {
        "Name": "http-data",
        "Driver": "local",
        "Mountpoint": "/home/user/.local/share/containers/storage/volumes/http-data/_data",
        "CreatedAt": "2022-07-12T17:10:12.709259987+02:00",
        "Labels": {},
        "Scope": "local",
        "Options": {}
    }
]

For rootless containers, Podman stores local volume data in the $HOME/.local/share/containers/storage/volumes/ directory.

To mount the volume into a container, refer to the volume by the volume name:

[user@host ~]$ podman run -p 8080:8080 --volume  http-data:/var/www/html \
  registry.access.redhat.com/ubi8/httpd-24:latest

Because Podman manages the volume, you do not need to configure SELinux permissions.
Exporting and Importing Data with Volumes

You can import data from a tar archive into an existing Podman volume by using the podman volume import VOLUME_NAME ARCHIVE_NAME command.

[user@host ~]$ podman volume import http_data web_data.tar.gz
...no output expected...

You can also export data from an existing Podman volume and save it as a tar archive on the local machine by using the podman volume export VOLUME NAME --output ARCHIVE_NAME command.

[user@host ~]$ podman volume export http_data --output web_data.tar.gz
...no output expected...

Storing Data with a tmpfs Mount

Some applications cannot use the default COW file system in a specific directory for performance reasons, but do not use persistence or data sharing for that directory.

For such cases, you can use the tmpfs mount type, which means that the data in a mount is ephemeral but does not use the COW file system:

[user@host ~]$ podman run -e POSTGRESQL_ADMIN_PASSWORD=redhat --network lab-net \
  --mount  type=tmpfs,tmpfs-size=512M,destination=/var/lib/pgsql/data \
  registry.redhat.io/rhel9/postgresql-13:1
]

For rootless containers, Podman stores local volume data in the $HOME/.local/share/containers/storage/volumes/ directory.

To mount the volume into a container, refer to the volume by the volume name:

[user@host ~]$ podman run -p 8080:8080 --volume  http-data:/var/www/html \
  registry.access.redhat.com/ubi8/httpd-24:latest

Because Podman manages the volume, you do not need to configure SELinux permissions.
Exporting and Importing Data with Volumes

You can import data from a tar archive into an existing Podman volume by using the podman volume import VOLUME_NAME ARCHIVE_NAME command.

[user@host ~]$ podman volume import http_data web_data.tar.gz
...no output expected...

You can also export data from an existing Podman volume and save it as a tar archive on the local machine by using the podman volume export VOLUME NAME --output ARCHIVE_NAME command.

[user@host ~]$ podman volume export http_data --output web_data.tar.gz
...no output expected...

Storing Data with a tmpfs Mount

Some applications cannot use the default COW file system in a specific directory for performance reasons, but do not use persistence or data sharing for that directory.

For such cases, you can use the tmpfs mount type, which means that the data in a mount is ephemeral but does not use the COW file system:

[user@host ~]$ podman run -e POSTGRESQL_ADMIN_PASSWORD=redhat --network lab-net \
  --mount  type=tmpfs,tmpfs-size=512M,destination=/var/lib/pgsql/data \
  registry.redhat.io/rhel9/postgresql-13:1